In [ ]:
import requests
import pandas as pd
import time
import datetime

BASE = 'https://api.kraken.com/0/public'

def kraken_ohlc(pair='XXBTZUSD', interval=60, since=None):
    params = {'pair': pair, 'interval': interval}
    if since is not None:
        params['since'] = since
    r = requests.get(f'{BASE}/OHLC', params=params)
    data = r.json()
    result = data['result']
    pair_key = [k for k in result if k != 'last'][0]
    bars = result[pair_key]
    last = result['last']
    df = pd.DataFrame(bars, columns=['timestamp','open','high','low','close','vwap','volume','trades'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)
    df = df.set_index('timestamp')
    return df, last

print('Ready.')

## Cell 2 — No `since` param (baseline)

In [ ]:
df, last = kraken_ohlc()
print(f'bars: {len(df)}')
print(f'first: {df.index[0]}')
print(f'last:  {df.index[-1]}')
print(f'last cursor: {last}  →  {datetime.datetime.utcfromtimestamp(last)} UTC')
df.head(3)

## Cell 3 — `since=0`

In [ ]:
df0, last0 = kraken_ohlc(since=0)
print(f'bars: {len(df0)}')
print(f'first: {df0.index[0]}')
print(f'last:  {df0.index[-1]}')
print(f'last cursor: {last0}  →  {datetime.datetime.utcfromtimestamp(last0)} UTC')
df0.head(3)

## Cell 4 — Pass the `last` cursor from Cell 3 as `since` (page 2)

In [ ]:
df1, last1 = kraken_ohlc(since=last0)
print(f'bars: {len(df1)}')
print(f'first: {df1.index[0]}')
print(f'last:  {df1.index[-1]}')
print(f'last cursor: {last1}  →  {datetime.datetime.utcfromtimestamp(last1)} UTC')
print(f'Did cursor advance? {last1 != last0}')
df1.head(3)

## Cell 5 — Try a real Unix timestamp (2020-01-01)

In [ ]:
since_2020 = int(datetime.datetime(2020, 1, 1).timestamp())
print(f'since_2020 = {since_2020}')
df2, last2 = kraken_ohlc(since=since_2020)
print(f'bars: {len(df2)}')
print(f'first: {df2.index[0]}')
print(f'last:  {df2.index[-1]}')
print(f'last cursor: {last2}  →  {datetime.datetime.utcfromtimestamp(last2)} UTC')
df2.head(3)

## Cell 6 — Try daily interval with since=0

In [ ]:
df_day, last_day = kraken_ohlc(interval=1440, since=0)
print(f'bars: {len(df_day)}')
print(f'first: {df_day.index[0]}')
print(f'last:  {df_day.index[-1]}')
print(f'last cursor: {last_day}  →  {datetime.datetime.utcfromtimestamp(last_day)} UTC')
df_day.head(3)

## Cell 7 — 2-page test loop (stops after 2 pages to save rate limit)

In [ ]:
all_frames = []
since = 0
prev_last = None

for i in range(4):  # try 4 pages max
    df_page, last_cursor = kraken_ohlc(since=since)
    all_frames.append(df_page)
    print(f'Page {i+1}: {len(df_page)} bars  {df_page.index[0].date()} → {df_page.index[-1].date()}  cursor={last_cursor}')
    
    if last_cursor == since or last_cursor == prev_last:
        print('Cursor did not advance — end of history')
        break
    
    prev_last = last_cursor
    since = last_cursor
    time.sleep(1)

combined = pd.concat(all_frames)
combined = combined[~combined.index.duplicated(keep='last')].sort_index()
print(f'\nTotal unique bars: {len(combined)}')
print(f'Date range: {combined.index[0].date()} → {combined.index[-1].date()}')